In [ ]:
# imdb top 1000 movies urls of all the pages (10)
url_temp = 'https://www.imdb.com/search/title/?groups=top_1000&view=simple&sort=user_rating,desc&count=100&start={}&ref_=adv_nxt'
urls = []
for i in range(1, 1001, 100):
    url = url_temp.format(str(i))
    urls.append(url)
urls

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

In [ ]:
# function to get page
def get_page(url):
    page = requests.get(url)
    return page

In [ ]:
# function to turn page into html content
def turn_to_soup(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [ ]:
# function to select all movies on the pages
def parse_soup(soup):
    href_titles = soup.select('div.col-title a')
    return href_titles

In [ ]:
# function to get title only from html
def get_titles(href_titles):
    titles = []
    for title in href_titles:
        titles.append(title.text)
    return titles

In [ ]:
# combine functions to iterate through all the page urls
all_titles = []

for url in urls:
    page = get_page(url)
    soup = turn_to_soup(page)
    href_titles = parse_soup(soup)
    titles = get_titles(href_titles)
    all_titles.extend(titles)

In [ ]:
all_titles

In [ ]:
# omdb api url with api key
api_movie_url = 'http://www.omdbapi.com/?apikey=669a3fed&'

# omdb params = title of movie
api_movie_params = []
for title in all_titles:
    api_movie_params.append({'t' : title})

In [ ]:
# function to make api call
def movie_call(movie_params):
    response = requests.get(api_movie_url, params=movie_params)
    movie_data = response.json()
    return movie_data

In [ ]:
# # try out movie_call function
# movie_call(api_movie_params[2])

In [ ]:
# function to parse movie data
def parse_movie_data(movie_data):
    parsed_movie_data = []
    for data in movie_data:
        if 'Title' not in data.keys():
            data.setdefault('Title', None)
        if 'Genre' not in data.keys():
            data.setdefault('Genre', None)
        if 'Director' not in data.keys():
            data.setdefault('Director', None)
        if 'Actors' not in data.keys():
            data.setdefault('Actors', None)
        if 'Metascore' not in data.keys():
            data.setdefault('Metascore', None)
        if 'imdbRating' not in data.keys():
            data.setdefault('imdbRating', None)
        if 'imdbID' not in data.keys():
            data.setdefault('imdbID', None)
        if 'Production' not in data.keys():
            data.setdefault('Production', None)
        mov_tuple = (data['Title'], data['Genre'], data['Director'], data['Actors'], data['Metascore'], data['imdbRating'], data['imdbID'], data['Production'])
        parsed_movie_data.append(mov_tuple)
    return parsed_movie_data

In [ ]:
# combine call and parse functions to loop through all movie titles
movie_data_total = []
for param in api_movie_params:
    movie_data = movie_call(param)
    movie_data_total.append(movie_data)
    parsed_movie_data = parse_movie_data(movie_data_total)

In [ ]:
# # print all parsed movie data
# parsed_movie_data

In [19]:
import mysql.connector 
import config

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.pw
)
cursor = cnx.cursor()


# # create new DB 
# cursor.execute("CREATE DATABASE movies")

cnx.database = 'movies'

In [ ]:
# create tables for the movies
# create_table_all = """
#     CREATE TABLE movies_all (
#     movieID int AUTO_INCREMENT,
#     title varchar(255),
#     genre varchar(255),
#     director varchar(255),
#     actors varchar(255),
#     metascore FLOAT,
#     imdb_rating FLOAT,
#     imdbID varchar(255),
#     production varchar(255),
#     PRIMARY KEY (movieID)
#     );"""

# create_table_movies = """
#     CREATE TABLE movies (
#     moviesID int AUTO_INCREMENT,
#     title varchar(255),
#     genre varchar(255),
#     imdbID varchar(255),
#     movieID int,
#     PRIMARY KEY (moviesID),
#     FOREIGN KEY (movieID) REFERENCES movies_all (movieID)
#     );"""

# create_table_production = """
#     CREATE TABLE production (
#     productionID int AUTO_INCREMENT,
#     director varchar(255),
#     actors varchar(255),
#     production varchar(255),
#     movieID int,
#     PRIMARY KEY (productionID),
#     FOREIGN KEY (movieID) REFERENCES movies_all (movieID)
#     );"""

# create_table_ratings = """
#     CREATE TABLE ratings (
#     ratingsID int AUTO_INCREMENT,
#     metascore FLOAT,
#     imdb_rating FLOAT,
#     movieID int,
#     PRIMARY KEY (ratingsID),
#     FOREIGN KEY (movieID) REFERENCES movies_all (movieID)
#     );"""

# create_table_movies_all2 = """
#     CREATE TABLE movies_all2 (
#     title varchar(100),
#     movieID2 int,
#     release_date DATE,
#     popularity FLOAT,
#     budget int,
#     genre varchar(100),
#     revenue FLOAT,
#     vote_avg FLOAT,
#     vote_count int,
#     imdbID varchar(50)
#     );"""

In [ ]:
create_table_movies_master = """
    CREATE TABLE movies_master (
    title varchar(100),
    movieID2 int,
    release_date DATE,
    popularity FLOAT,
    budget int,
    genre varchar(100),
    revenue FLOAT,
    vote_avg FLOAT,
    vote_count int,
    imdbID varchar(50)
    );"""

In [ ]:
# # create tables in DB

# cursor.execute(create_table_all)
# cursor.execute(create_table_movies)
# cursor.execute(create_table_production)
# cursor.execute(create_table_ratings)
# cursor.execute(create_table_movies_all2)

In [ ]:
# write function to input all data into DB

insert_all_data = """
    INSERT INTO movies_all (
    title,
    genre,
    director,
    actors,
    metascore,
    imdb_rating,
    imdbID,
    production)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"""

In [ ]:
# # insert data into all data table
# cursor.executemany(insert_all_data, parsed_movie_data)
# cnx.commit()

In [ ]:
# write a function to take parsed data and insert it into DB

insert_movies = """
    INSERT INTO movies (
    title, genre, imdbID, movieID)
    VALUES (%s, %s, %s, %s);"""

insert_production = """
    INSERT INTO production (
    director, actors, production, movieID)
    VALUES (%s, %s, %s, %s);"""

insert_ratings = """
    INSERT INTO ratings (
    metascore, imdb_rating, movieID)
    VALUES (%s, %s, %s);"""

In [ ]:
# # insert data into movies table
# movies_table_data = cursor.execute("""SELECT title, genre, imdbID, movieID FROM movies_all;""")
# cursor.executemany(insert_movies, cursor.fetchall())
# cnx.commit()

In [ ]:
# # insert data into producion table
# production_table_data = cursor.execute("""SELECT director, actors, production, movieID FROM movies_all;""")
# cursor.executemany(insert_production, cursor.fetchall())
# cnx.commit()

In [ ]:
# # insert data into ratings table
# ratings_table_data = cursor.execute("""SELECT metascore, imdb_rating, movieID FROM movies_all;""")
# cursor.executemany(insert_ratings, cursor.fetchall())
# cnx.commit()

In [17]:
import pandas as pd

In [20]:
#join master tables
cursor.execute("""
    SELECT *
    FROM movies.movies_all a
    JOIN movies.movies_all2 b
    ON a.imdbID = b.imdbID;""")

#create dataframe for master table
movies_all = cursor.fetchall()
df = pd.DataFrame(movies_all)
df.columns = [i[0] for i in cursor.description]
df = df.set_index('movieID')

In [21]:
cnx.commit()

In [26]:
df.columns

Index(['title', 'genres', 'director', 'actors', 'metascore', 'imdb_rating',
       'imdbID', 'production', 'title', 'movieID2', 'release_date',
       'popularity', 'budget', 'genre', 'revenue', 'vote_avg', 'vote_count',
       'imdbID'],
      dtype='object')